In [43]:
import librosa
import matplotlib.pyplot as plt
import librosa.display
import os
import numpy as np
from IPython.display import Audio
import wave
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
root_fp = "../../DATA/ml-stuttering-events-dataset/"
CLIPS_DIR = "/home/alien/Git/DATA/ml-stuttering-events-dataset/clips/"
# CLIPS_DIR = "/home/alien/Git/DATA/ml-stuttering-events-dataset/CLIP_DIR/"
df = pd.read_csv(root_fp + "SEP-28k_labels.csv")
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [45]:
df.shape

(28177, 17)

In [46]:
df['Name'] = df[df.columns[0:3]].apply(
    lambda x: '_'.join(x.dropna().astype(str)),
    axis=1
)
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech,Name
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0,HeStutters_0_0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0,HeStutters_0_1
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0,HeStutters_0_2
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0,HeStutters_0_3
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0,HeStutters_0_4


In [47]:
df['Name']

0                HeStutters_0_0
1                HeStutters_0_1
2                HeStutters_0_2
3                HeStutters_0_3
4                HeStutters_0_4
                  ...          
28172    WomenWhoStutter_109_35
28173    WomenWhoStutter_109_36
28174    WomenWhoStutter_109_37
28175    WomenWhoStutter_109_38
28176    WomenWhoStutter_109_39
Name: Name, Length: 28177, dtype: object